#6.2　《分析》データを利用して退会者の特徴としきい値を理解しよう
## 6.2.2　データの理解

#### コード6.1

In [ ]:
! pip install japanize-matplotlib

#### コード6.2

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import japanize_matplotlib
from sklearn import tree
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier

#### コード6.3

In [ ]:
ct = pd.read_csv('GymCustomers.csv')

#### コード6.4

In [ ]:
ct.head()

#### コード6.5

In [ ]:
ct.shape

#### コード6.6

In [ ]:
ct.isnull().sum()

#### コード6.7

In [ ]:
ct.describe()

#### コード6.8

In [ ]:
categorical_columns = ct.select_dtypes(include=['object', 'bool']).columns

for col in categorical_columns:
    print(col, ct[col].unique())

#### コード6.9

In [ ]:
for col in categorical_columns:
    unique_counts = ct[col].value_counts()
    plt.bar(unique_counts.index, unique_counts.values)
    plt.xlabel(col)
    plt.ylabel("Count")
    plt.title(f"Unique Value Counts for {col}")
    plt.show()

#### コード6.10

In [ ]:
ct.groupby('年収')[['年齢', '利用年数', '退会フラグ']].mean()

In [ ]:
ct.groupby('浄水サブスクリプション')[['年齢', '利用年数', '退会フラグ']].mean()

In [ ]:
ct.groupby('SNSアカウントフォロー')[['年齢', '利用年数', '退会フラグ']].mean()

In [ ]:
ct.groupby('利用年数')[['年齢', '利用年数', '退会フラグ']].mean()

In [ ]:
ct.groupby('ジムポイント会員')[['年齢', '利用年数', '退会フラグ']].mean()

### 6.2.3　データの前処理

#### コード6.11

In [ ]:
ct['ジムポイント会員'].fillna('No', inplace=True)

#### コード6.6（再度実行）

In [ ]:

ct.isnull().sum()

#### 文字列データを数値データに変換する

#### コード6.12

In [ ]:
ct.dtypes

#### コード6.13

In [ ]:
ct_encoding = ct.copy()

#### コード6.14

In [ ]:
categorical_columns = ct_encoding.select_dtypes(include=['object', 'bool']).columns

for col in categorical_columns:
    print(col, ct_encoding[col].unique())

#### コード6.15

In [ ]:
encoding_columns = ['ジムポイント会員', 'SNSアカウントフォロー', '浄水サブスクリプション']

for column in encoding_columns:
    ct_encoding[column] = ct_encoding[column].apply(lambda x: 1 if x == 'Yes' else 0)

ct_encoding.head()

# 無名関数lambdaを利用しない場合
# def yes_no_to_binary(x):
#     if x == 'Yes':
#         return 1
#     else:
#         return 0


# encoding_columns = ['ジムポイント会員', 'SNSアカウントフォロー', '浄水サブスクリプション']


# for column in encoding_columns:
#   ct_encoding[column] = ct_encoding[column].apply(yes_no_to_binary)

#### コード6.16

In [ ]:
def income_category_to_numeric(category):
    if category == "低所得":
        return 1
    elif category == "中所得":
        return 2
    elif category == "高所得":
        return 3
    else:
        print('予期せぬ値')

ct_encoding['年収'] = ct_encoding['年収'].apply(income_category_to_numeric)

In [ ]:
ct_encoding.head()

#### コード6.17

In [ ]:
X = ct_encoding.drop('退会フラグ', axis=1)
y = ct_encoding['退会フラグ']

### 6.2.4　モデリング 〜決定木による要因把握〜

#### コード6.18

In [ ]:
dtree = DecisionTreeClassifier(max_depth=2)
dtree.fit(X, y)

### 精度評価指標について

#### コード6.19


In [ ]:
from sklearn.model_selection import train_test_split

# 説明変数と目的変数に分割
X = ct_encoding.drop('退会フラグ', axis=1)
y = ct_encoding['退会フラグ']

# train_test_splitで分割
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=0)

In [ ]:
# 分割後のデータ数
print(X_train.shape)
print(X_val.shape)
print(y_train.shape)
print(y_val.shape)

#### コード6.20


In [ ]:
dtree = DecisionTreeClassifier(random_state=0)
dtree.fit(X_train, y_train)

#### コード6.21


In [ ]:
y_pred = dtree.predict_proba(X_val)
y_pred

#### コード6.22


In [ ]:
from sklearn.metrics import confusion_matrix

# しきい値0.5よりも大きければ1
y_pred_flag = (y_pred[:, 1] > 0.5).astype(int)
cm = confusion_matrix(y_val, y_pred_flag)
pd.crosstab(y_val, y_pred_flag, rownames=['Actual'], colnames=['Predicted'])

#### コード6.23


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, precision_recall_curve

accuracy = accuracy_score(y_val, y_pred_flag)
precision = precision_score(y_val, y_pred_flag)
recall = recall_score(y_val, y_pred_flag)
f1 = f1_score(y_val, y_pred_flag)

print("正解率:", accuracy)
print("適合率:", precision)
print("再現率:", recall)
print("F１スコア:", f1)

#### コード6.24


In [ ]:
from sklearn.metrics import precision_recall_curve, auc
precision, recall, thresholds = precision_recall_curve(y_val, y_pred[:, 1])
auc = auc(recall, precision)
print(auc)

plt.plot(recall, precision, label='PR curve (area = %.2f)'%auc)
plt.legend()
plt.title('PR curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.grid(True)
plt.show()

#### コード6.25


In [ ]:
dtree = DecisionTreeClassifier(max_depth=3, random_state=0)
dtree.fit(X, y)

plt.figure(figsize=(30,10))
tree.plot_tree(dtree, filled=True, feature_names=X.columns, class_names=['継続', '退会'])
plt.show()